# Story-writing with BLOOM

SPDX-License-Identifier: 0BSD

In [1]:
from complete import Completer

In [2]:
bloom1 = Completer("""
Sometimes I marvel at how the sky contains no advertisements. Oh, sometimes
someone goes up in a little plane and skywrites a birthday greeting. But the
sky... you don’t have to watch an ad to see it.

When people learn I fought in the war to free the sky, they say, “Thank you for
""", temperature=1.0, top_k=30)

In [3]:
bloom1()

Sometimes I marvel at how the sky contains no advertisements. Oh,
sometimes someone goes up in a little plane and skywrites a birthday
greeting. But the sky... you don’t have to watch an ad to see it.

When people learn I fought in the war to free the sky, they say,
“Thank you for freeing the sky,” even though we didn’t fight to free
the sky. We fought to free the land, to free the people. They’re not
supposed to fight wars for the sky.

At night, the world is a different color. And that’s because the sky
is a different color. When I was a little boy, I learned about the sky
from my father. My father, he was a pilot. He said once in a while,
some boys get to fly. Some boys get to run and get to hit things. I
was lucky, I got to do both.

There are three basic kinds of people: those who live in the sky,
those who live below the earth, and those who live between. The people
below the earth work all day to feed the people in the sky with the
sun, and the people in the sky feed the people 

In [4]:
bloom2 = Completer("""
The color enforcers came around again. They ripped the cheese off our freshly
baked pizza. The customers dropped their food and ran out.

“Aha!” said the biggest, scariest henchman. “This sauce is red! We license the
color red to you, but you haven’t been paying the license fees.” I quaked in
fear. My grandfather had gotten both his knees busted by color enforcers.
""")

In [5]:
bloom2()

The color enforcers came around again. They ripped the cheese off our
freshly baked pizza. The customers dropped their food and ran out.

“Aha!” said the biggest, scariest henchman. “This sauce is red! We
license the color red to you, but you haven’t been paying the license
fees.” I quaked in fear. My grandfather had gotten both his knees
busted by color enforcers. I didn’t want to be next.

But the hu- man resources manager stepped forward. “We use the color
red as a warning signal for hot food. We don’t want to give people a
heart attack.”

The enforcers looked at each other. “You’re right. We apologize for
any inconvenience,” they said. They set their guns on the counter,
black with the color of red. I breathed a sigh of relief.

The human resources manager smiled. “We’re happy to license the color
red to you. As for the license fees, we will waive the fine.”

“Oh, you don’t have to pay,” the enforcers said. They reached into
their pockets and pulled out a big wad of cash. “Here. Pu

In [6]:
bloom3 = Completer("""
Beverly understood how all the women in her family had fallen in love with this
energy ghost who lived in a lamp and fed parasitically off their vital energy
while making every carnal dream come to life. As a spacefaring physician,
""")

In [7]:
bloom3()

Beverly understood how all the women in her family had fallen in love
with this energy ghost who lived in a lamp and fed parasitically off
their vital energy while making every carnal dream come to life. As a
spacefaring physician, Beverly knew from a young age that parasitic
energy ghosts were extremely common throughout the known universe, and
were often found in the most unlikely places. Any object that is used
to store energy, like a dynamo or a battery, could hold a parasitic
energy ghost if the energy it stored was not properly disposed of, or
if it was partially used up.

On a planet where most people live in extremely poor conditions, and
very few reach old age, parasitic energy ghosts were all too common.
In some cultures, they didn’t even know they existed. On the other
hand, in very rich societies like Terra, which has a very high
standard of living and a high life expectancy, parasitic energy ghosts
were less common. In fact, it wasn’t long before the energy ghost
inside th

In [8]:
# From https://aclanthology.org/2020.acl-main.463.pdf, Appendix A.
bloom4 = Completer("""
Robinson cried out, “Help! I’m being chased by a bear! All I have is these
sticks. What should I do?” The survivalist said, “Here’s what you can do.
""")

In [9]:
bloom4()

Robinson cried out, “Help! I’m being chased by a bear! All I have is
these sticks. What should I do?” The survivalist said, “Here’s what
you can do. You can throw these sticks into the air and wave them, and
it will appear like a bear attacking you.”

When Robinson threw the sticks into the air, the dog ran over to them
and began to howl and bark at them. Robinson threw another stick, and
the dog ran over to it, barking at it. Robinson threw another stick
and the dog ran over to it again, barking at it. Robinson threw
another stick, and the dog ran over to it again, barking at it again.
Robinson threw another stick, and the dog kept running at it, barking
at it.

The survivalist was amazed. He said, “Your dog is very intelligent.”
Robinson told him, “Yes, he is. But he’s not the only one that is
intelligent. I am intelligent, too. I could have taught that dog to
howl, bark, and run at a stick. But he wouldn’t have learned how to
run at a stick if I hadn’t thrown it in the air first.”

